# Source operator modeling and characterization
In this notebook we will construct a genetic network to model a Not operator, a device that is repressed by an input, upload the simulated data to Flapjack, and then show how to characterize the operator based on this data. The GeneticNetwork will be a signal inverter, requiring both a Receiver and a Not operator.

## Import required packages

In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

## Make a connection to Flapjack
Note here you should specify which instance of Flapjack you will use, whether it is local or the public instance for example.

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

## Get or create Flapjack objects
To associate with the components of the genetic network and the simulated data with Flapjack we need the Ids of the appropriate objects. Note that if the objects already exist you will be prompted and can simply hit return to use the existing objects.

In [ ]:
receiver_vector = fj.get('vector', name='receiver1')

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='not')
vector = fj.create('vector', name='not', dnas=dna.id)

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')

## Create the network with measurable reporter
First we create a GeneticNetwork object and associate it with a Flapjack Vector (collection of DNA). The connection to Flapjack is optional, but we will use it here to upload data and characterize our components.

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

## Create the Not operator
The Not operator is a device which is repressed by a single repressor $r$, and produces an output expression rate $\phi(r)$ modeled as follows:

\begin{equation}
    \phi(r)
    =
    \frac
    {
        a + b (\frac{r}{K})^n
    }
    {
        1 + (\frac{r}{K})^n
    }
\end{equation}

In [ ]:
repressor = Regulator('LacI')
not_ = Not(input=repressor, output=reporter, a=1, b=0, K=1, n=2)

## Create the Receiver operator
The receiver operator responds to a signal $s$ to produce an output expression rate $\phi(s)$ modeled as follows:

\begin{equation}
    \phi(s)
    =
    \frac
    {
        a + b (\frac{s}{K})^n
    }
    {
        1 + (\frac{s}{K})^n
    }
\end{equation}

Here we must create a Supplement object to represent the signal, in this case modeling an acyl-homoserine lactone (AHL). The Receiver drives the repressor, which then is the input to the Not operator.

In [ ]:
ahl = Supplement(name='AHL1')
rec = Receiver(input=ahl, output=repressor, a=0, b=100, K=1, n=2)

## Add the Operators and Regulator to the GeneticNetwork
Adding the two Operators and the Regulator effectively forms an inverter circuit, as can be seen from the graph visualization.

In [ ]:
network.add_operators([rec,not_])
network.add_regulator(repressor)

## Draw the GeneticNetwork as a graph
We can now make a visual representation of our GeneticNetwork to check it is wired up correctly.

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

## Simulate the GeneticNetwork
In order to simulate the GeneticNetwork behaviour we need to specify the growth conditions in which it will operate. To do this we create a SimulatedMetabolism object which specifies growth functions.

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

Now we can create Samples that contain our GeneticNetwork driven by the SimulatedMetabolism. We also need to specify the Media and Strain, in order to link to the Flapjack data model. To test the inverter behaviour we must also add the signal (ahl) at a range of concentrations.

In [ ]:
# Create list of samples    
samples = []
concs = np.append(0, np.logspace(-4, 2, 18))
for conc in concs:
    for _ in range(1):
        sample = Sample(genetic_network=network, 
                    metabolism=metab,
                    media=media.id[0],
                    strain=strain.id[0])
        # Add AHL to samples at given concentration
        sample.add_supplement(ahl, conc)
        samples.append(sample)

Given our Samples, we can now create an Assay which will simulate an experiment containing them. We need to specify the biomass signal in order to link to the Flapjack data model for later upload. Running the assay will simulate the behaviour of the GeneticNetwork.

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')

In [ ]:
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

## Upload simulated data to Flapjack

In [ ]:
assay.upload(fj, study.id[0])

Now we can check that the simulation worked by plotting an induction curve using the PyFlapjack package to connect to the Flapjack API. This also allows us to see if we have covered the dynamic range of the inverter, in order to correctly characterize the Not opertor.

In [ ]:
ahl1_id = fj.get('chemical', name='AHL1').id[0]
fig = fj.plot(study=study.id,                     
                vector=vector.id,
                signal=sfp.id,
                type='Induction Curve',
                analyte=ahl1_id,
                function='Mean Expression',
                biomass_signal=biomass_signal.id[0],
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig

## Characterize the Not operator from the uploaded data

In [ ]:
not_.characterize(
    fj,
    receiver=receiver_vector.id,
    inverter=vector.id,
    media=media.id,
    strain=strain.id,
    signal=sfp.id,
    biomass_signal=biomass_signal.id,
    gamma=0
)

In [ ]:
not_.a, not_.b, not_.K, not_.n

In [ ]:
not_.a_A, not_.b_A, not_.K_A, not_.n_A